# <font size="6">Libraries</font>

In [1]:
import pandas as pd
import numpy as np

# <font size="6">dataset</font>

In [2]:
train_df = pd.read_parquet('data/train.parquet').drop(columns=['transaction_number',
                                                               'reversed_transaction', 
                                                               'transaction_max',
                                                               'time'])

valid_df = pd.read_parquet('data/valid.parquet').drop(columns=['transaction_number',
                                                               'reversed_transaction', 
                                                               'transaction_max',
                                                               'time'])

In [3]:
train_df['mcc_next'] = train_df.groupby(['app_id'])['mcc'].shift(-1)
train_df['amnt_next'] = train_df.groupby(['app_id'])['amnt'].shift(-1)
train_df['hour_diff_next'] = train_df.groupby(['app_id'])['hour_diff'].shift(-1)
train_df['amnt_bins_next'] = train_df.groupby(['app_id'])['amnt_bins'].shift(-1)
train_df['amnt_mcc_bins_next'] = train_df.groupby(['app_id'])['amnt_mcc_bins'].shift(-1)

valid_df['mcc_next'] = valid_df.groupby(['app_id'])['mcc'].shift(-1)
valid_df['amnt_next'] = valid_df.groupby(['app_id'])['amnt'].shift(-1)
valid_df['hour_diff_next'] = valid_df.groupby(['app_id'])['hour_diff'].shift(-1)
valid_df['amnt_bins_next'] = valid_df.groupby(['app_id'])['amnt_bins'].shift(-1)
valid_df['amnt_mcc_bins_next'] = valid_df.groupby(['app_id'])['amnt_mcc_bins'].shift(-1)

In [4]:
last_transactions_train = train_df.groupby('app_id').tail(1)

train_df = train_df.drop(last_transactions_train.index)

train_df = train_df.dropna()

In [5]:
first_transactions_valid = valid_df.groupby('app_id').head(1)
first_transactions_valid = first_transactions_valid.dropna()

# <font size="6">Model</font>

In [6]:
last_transactions_train

,app_id,amnt,mcc,hour_diff,amnt_bins,amnt_mcc_bins,mcc_next,amnt_next,hour_diff_next,amnt_bins_next,amnt_mcc_bins_next
130,0,0.348838,2,42,3,8,NaN,NaN,NaN,NaN,NaN
436,1,0.297589,20,4,0,77,NaN,NaN,NaN,NaN,NaN
615,2,0.402386,2,31,3,8,NaN,NaN,NaN,NaN,NaN
682,4,0.621777,9,26,3,36,NaN,NaN,NaN,NaN,NaN
892,6,0.333474,3,3,2,11,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95045500,1003041,0.299564,4,0,0,13,NaN,NaN,NaN,NaN,NaN
95045710,1003044,0.349617,4,0,3,16,NaN,NaN,NaN,NaN,NaN
95045920,1003047,0.229750,18,5,0,69,NaN,NaN,NaN,NaN,NaN
95046089,1003048,0.662458,2,41,3,8,NaN,NaN,NaN,NaN,NaN


In [8]:
last_transactions_train = last_transactions_train.rename(columns={'amnt': 'amnt_pred',
                                                        'mcc': 'mcc_pred',
                                                        'hour_diff': 'hour_diff_pred',
                                                        'amnt_mcc_bins': 'amnt_mcc_bins_pred',
                                                        'amnt_bins': 'amnt_bins_pred'}).drop(columns=['hour_diff_next',
                                                                                                              'amnt_next',
                                                                                                              'amnt_mcc_bins_next',
                                                                                                              'mcc_next',
                                                                                                               'amnt_bins_next'])

In [9]:
dataframe = last_transactions_train.merge(first_transactions_valid, on='app_id', how='inner')

# <font size="6">Metrics</font>

In [11]:
from sklearn.metrics import mean_absolute_error, accuracy_score

print('Mae score:', mean_absolute_error(dataframe['hour_diff'], dataframe['hour_diff_pred']))
print('Type accuracy:', accuracy_score(dataframe['amnt_mcc_bins'], dataframe['amnt_mcc_bins_pred']))
print('Amnt accuracy:', accuracy_score(dataframe['amnt_bins'], dataframe['amnt_bins_pred']))
print('MCC accuracy:', accuracy_score(dataframe['mcc'], dataframe['mcc_pred']))

Mae score: 44.63902170258862
Type accuracy: 0.1733148743042845
Amnt accuracy: 0.48457752792200515
MCC accuracy: 0.285572541182623
